# About
* **Author**: Adil Rashitov
* **Created at**: 21.06.2021
* **Goal**: perform test web scrapping for [118 direct](http://www.118.direct)
* **Deliverable**: Prototype of web scrapper

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False


import requests
from bs4 import BeautifulSoup
import re
import time
import sqlalchemy

# GLOBAL VARS
from dotenv import load_dotenv
load_dotenv('./.rds_endpoint')

True

# Data

* Web scrapper parameters

In [4]:
TABLE_NAME="WEB_SCRAPPER_PARAMETERS"


RDS_ENGINE = sqlalchemy.create_engine(os.environ['RDS_ENDPOINT'])
pd.read_sql(f'select * from public."{TABLE_NAME}"', con=RDS_ENGINE)

,scrapping_parameter_id,what,where,n_entities_at_category_at_location,n_pages_at_category_at_location,page,base_url,full_url
0,0,Abattoirs,Manchester,4,1,1,http://www.118.direct/listing/search?what=Abat...,http://www.118.direct/listing/search?what=Abat...
1,1,Access Control Systems,Manchester,1,1,1,http://www.118.direct/listing/search?what=Acce...,http://www.118.direct/listing/search?what=Acce...
2,2,Access Equipment,Manchester,17,2,1,http://www.118.direct/listing/search?what=Acce...,http://www.118.direct/listing/search?what=Acce...
3,3,Access Equipment,Manchester,17,2,2,http://www.118.direct/listing/search?what=Acce...,http://www.118.direct/listing/search?what=Acce...
4,4,Accountants,Manchester,722,49,1,http://www.118.direct/listing/search?what=Acco...,http://www.118.direct/listing/search?what=Acco...
...,...,...,...,...,...,...,...,...
6633,6633,Wrought Ironwork,Manchester,28,2,2,http://www.118.direct/listing/search?what=Wrou...,http://www.118.direct/listing/search?what=Wrou...
6634,6634,Yacht Brokers,Manchester,1,1,1,http://www.118.direct/listing/search?what=Yach...,http://www.118.direct/listing/search?what=Yach...
6635,6635,Yoga,Manchester,26,2,1,http://www.118.direct/listing/search?what=Yoga...,http://www.118.direct/listing/search?what=Yoga...
6636,6636,Yoga,Manchester,26,2,2,http://www.118.direct/listing/search?what=Yoga...,http://www.118.direct/listing/search?what=Yoga...
